In [10]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from typing import List, Dict, Any, Tuple
import uuid
import random
from dataclasses import dataclass, field
import os
import warnings

class SemanticaPrimitive:
    """
    Represents a semantic primitive with rich contextual metadata
    """
    def __init__(self, 
                 value: str, 
                 primitive_type: str = 'generic', 
                 language: str = 'universal',
                 embedding: np.ndarray = None):
        self.id = str(uuid.uuid4())
        self.value = value
        self.type = primitive_type
        self.language = language
        self.embedding = embedding if embedding is not None else np.random.rand(50)
        self.metadata = {}
        self.semantic_energy = 0.5  # New attribute to represent semantic potency

    def __repr__(self):
        return f"Primitive(value={self.value}, type={self.type}, lang={self.language})"

@dataclass
class SemanticRelation:
    """
    Represents a relation between primitives with validation and confidence
    """
    source: SemanticaPrimitive
    target: SemanticaPrimitive
    relation_type: str
    confidence: float = 0.0
    agent_validations: List[str] = field(default_factory=list)
    semantic_intensity: float = 0.0
    
    def validate(self, agent_id: str):
        """Track agent validations and update confidence"""
        if agent_id not in self.agent_validations:
            self.agent_validations.append(agent_id)
            self.confidence = min(1.0, self.confidence + 0.1)
            self.semantic_intensity = len(self.agent_validations) * 0.2

class SemanticAgent:
    """
    An agent that can propose, validate, and modify semantic relations
    """
    def __init__(self, agent_id: str, expertise: Dict[str, float] = None):
        self.id = agent_id
        self.expertise = expertise or {}
        self.memory_graph = nx.DiGraph()

    def propose_relation(self, 
                         source: SemanticaPrimitive, 
                         target: SemanticaPrimitive, 
                         relation_type: str) -> SemanticRelation:
        """Propose a semantic relation based on agent's expertise"""
        confidence = self.expertise.get(relation_type, 0.5)
        relation = SemanticRelation(source, target, relation_type, confidence)
        relation.validate(self.id)
        
        # Boost semantic energy of primitives
        source.semantic_energy += 0.1
        target.semantic_energy += 0.1
        
        return relation

class SemanticaGraph:
    """
    Central semantic graph that manages primitives, relations, and convergence
    """
    def __init__(self, output_dir: str = None):
        self.primitives = {}
        self.relations = []
        self.agents = {}
        self.convergence_graph = nx.DiGraph()
        self.output_dir = output_dir or os.path.join(os.getcwd(), 'output')
        os.makedirs(self.output_dir, exist_ok=True)

    def add_primitive(self, primitive: SemanticaPrimitive):
        """Add a new primitive to the semantic space"""
        self.primitives[primitive.id] = primitive
        return primitive

    def add_agent(self, agent: SemanticAgent):
        """Register a new semantic agent"""
        self.agents[agent.id] = agent

    def propose_relation(self, 
                         source_primitive: SemanticaPrimitive, 
                         target_primitive: SemanticaPrimitive, 
                         relation_type: str,
                         proposing_agent: SemanticAgent):
        """Propose and potentially validate a semantic relation"""
        relation = proposing_agent.propose_relation(
            source_primitive, 
            target_primitive, 
            relation_type
        )
        
        # Cross-validate with other agents
        for agent in self.agents.values():
            if agent.id != proposing_agent.id:
                # More nuanced validation logic
                validation_prob = 0.7 * (1 + agent.expertise.get(relation_type, 0.5))
                if random.random() < validation_prob:
                    relation.validate(agent.id)
        
        self.relations.append(relation)
        
        # Update convergence graph
        self.convergence_graph.add_edge(
            source_primitive.id, 
            target_primitive.id, 
            relation_type=relation_type,
            confidence=relation.confidence,
            semantic_intensity=relation.semantic_intensity
        )

    def animate_convergence(self, num_frames: int = 100):
        """
        Create an animated visualization of semantic convergence
        """
        # Debugging: Check the state of primitives, relations, and graph
        print(f"Number of primitives: {len(self.primitives)}")
        print(f"Number of relations: {len(self.relations)}")
        print(f"Number of nodes in convergence graph: {self.convergence_graph.number_of_nodes()}")
        print(f"Number of edges in convergence graph: {self.convergence_graph.number_of_edges()}")

        fig, ax = plt.subplots(figsize=(15, 10), facecolor='black')
        plt.style.use('dark_background')

        def update(frame):
            ax.clear()
            ax.set_facecolor('black')
            plt.title("Semantic Convergence Dynamics", color='white', fontsize=16)

            # Periodically add new relations or validate existing ones
            if frame % 10 == 0:
                # Check if there are agents and primitives to work with
                if not self.agents:
                    warnings.warn("No agents available. Skipping agent-related logic.")
                    return
                if not self.primitives:
                    warnings.warn("No primitives available. Skipping primitive-related logic.")
                    return

                # Simulate new relation proposals or validations
                for _ in range(2):
                    source = random.choice(list(self.primitives.values()))
                    target = random.choice(list(self.primitives.values()))
                    agent = random.choice(list(self.agents.values()))
                    self.propose_relation(source, target, 'exploration', agent)

            # Prepare graph layout
            pos = nx.spring_layout(self.convergence_graph, k=0.5, iterations=50)

            # Node colors and sizes based on semantic energy
            node_colors = [self.primitives[node].semantic_energy for node in self.convergence_graph.nodes()]
            node_sizes = [100 + self.primitives[node].semantic_energy * 500 for node in self.convergence_graph.nodes()]

            # Edge weights and colors based on relation confidence
            edge_weights = [
                self.convergence_graph[u][v].get('confidence', 0.1) * 5 
                for (u, v) in self.convergence_graph.edges()
            ]
            edge_colors = [
                plt.cm.plasma(self.convergence_graph[u][v].get('confidence', 0.1)) 
                for (u, v) in self.convergence_graph.edges()
            ]

            # Draw nodes
            nx.draw_networkx_nodes(
                self.convergence_graph, 
                pos, 
                node_color=node_colors, 
                node_size=node_sizes,
                cmap=plt.cm.viridis,
                alpha=0.8
            )

            # Draw edges
            nx.draw_networkx_edges(
                self.convergence_graph, 
                pos, 
                width=edge_weights,
                edge_color=edge_colors,
                alpha=0.6,
                arrows=True,
                connectionstyle='arc3,rad=0.1'
            )

            # Draw labels
            nx.draw_networkx_labels(
                self.convergence_graph, 
                pos, 
                labels={node: self.primitives[node].value for node in self.convergence_graph.nodes()},
                font_size=8,
                font_color='white'
            )

            # Add frame number and global semantic energy
            total_semantic_energy = sum(p.semantic_energy for p in self.primitives.values())
            ax.text(0.02, 0.98, f"Frame: {frame} / {num_frames}", transform=ax.transAxes, color='white', fontsize=10, verticalalignment='top')
            ax.text(0.02, 0.93, f"Total Semantic Energy: {total_semantic_energy:.2f}", transform=ax.transAxes, color='white', fontsize=10, verticalalignment='top')

            plt.axis('off')

        # Create animation
        anim = animation.FuncAnimation(fig, update, frames=num_frames, interval=200)
        
        # Save the animation
        output_path = os.path.join(self.output_dir, f'semantic_convergence_{uuid.uuid4()}.gif')
        anim.save(output_path, writer='pillow', fps=5)
        plt.close(fig)
        
        print(f"Animation saved to {output_path}")
        return output_path

# Phase 1

In [ ]:

# Demonstration of Semantica principles with expanded vocabulary and relations
def semantic_convergence_demo(output_dir: str = None):
    # Initialize Semantica Graph
    semantica = SemanticaGraph(output_dir)
    
    # Create expanded primitives with richer context
    primitives = [
        # Fruits
        SemanticaPrimitive("apple", primitive_type="fruit", language="english"),
        SemanticaPrimitive("Apfel", primitive_type="fruit", language="german"),
        SemanticaPrimitive("manzana", primitive_type="fruit", language="spanish"),
        
        # Colors
        SemanticaPrimitive("red", primitive_type="color", language="english"),
        SemanticaPrimitive("rot", primitive_type="color", language="german"),
        SemanticaPrimitive("rojo", primitive_type="color", language="spanish"),
        
        # Tastes
        SemanticaPrimitive("sweet", primitive_type="taste", language="english"),
        SemanticaPrimitive("süß", primitive_type="taste", language="german"),
        SemanticaPrimitive("dulce", primitive_type="taste", language="spanish"),
        
        # Additional fruits
        SemanticaPrimitive("orange", primitive_type="fruit", language="english"),
        SemanticaPrimitive("Orange", primitive_type="fruit", language="german"),
        SemanticaPrimitive("naranja", primitive_type="fruit", language="spanish"),
        
        # Size attributes
        SemanticaPrimitive("big", primitive_type="size", language="english"),
        SemanticaPrimitive("small", primitive_type="size", language="english"),
        
        # Abstract concepts
        SemanticaPrimitive("health", primitive_type="concept", language="english"),
        SemanticaPrimitive("nutrition", primitive_type="concept", language="english"),
        
        # Food categories
        SemanticaPrimitive("fruit", primitive_type="category", language="english"),
        SemanticaPrimitive("dessert", primitive_type="category", language="english"),
        
        # Seasons
        SemanticaPrimitive("autumn", primitive_type="season", language="english"),
        SemanticaPrimitive("summer", primitive_type="season", language="english")
    ]
    
    # Add primitives to the graph
    primitive_dict = {}  # For easy reference later
    for p in primitives:
        semantica.add_primitive(p)
        primitive_dict[p.value] = p
    
    # Create agents with different expertise profiles
    agents = [
        SemanticAgent("linguist1", {"translation": 0.9, "similarity": 0.7, "exploration": 0.6, "categorization": 0.5}),
        SemanticAgent("linguist2", {"translation": 0.8, "similarity": 0.6, "exploration": 0.5, "categorization": 0.7}),
        SemanticAgent("translator", {"translation": 1.0, "similarity": 0.9, "exploration": 0.8, "categorization": 0.4}),
        SemanticAgent("nutritionist", {"health_relation": 0.9, "categorization": 0.8, "similarity": 0.5, "exploration": 0.7}),
        SemanticAgent("chef", {"taste_relation": 0.9, "similarity": 0.8, "categorization": 0.9, "exploration": 0.8})
    ]
    
    # Add agents to the graph
    for agent in agents:
        semantica.add_agent(agent)
    
    # Propose initial relations - translations between languages
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["Apfel"],
        "translation",
        agents[0]
    )
    
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["manzana"],
        "translation",
        agents[2]
    )
    
    semantica.propose_relation(
        primitive_dict["red"],
        primitive_dict["rot"],
        "translation",
        agents[1]
    )
    
    semantica.propose_relation(
        primitive_dict["red"],
        primitive_dict["rojo"],
        "translation",
        agents[2]
    )
    
    semantica.propose_relation(
        primitive_dict["sweet"],
        primitive_dict["süß"],
        "translation",
        agents[0]
    )
    
    semantica.propose_relation(
        primitive_dict["sweet"],
        primitive_dict["dulce"],
        "translation",
        agents[2]
    )
    
    # Descriptive relations - attributes of objects
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["red"],
        "has_attribute",
        agents[0]
    )
    
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["sweet"],
        "has_attribute",
        agents[4]  # chef
    )
    
    semantica.propose_relation(
        primitive_dict["orange"],
        primitive_dict["sweet"],
        "has_attribute",
        agents[4]  # chef
    )
    
    # Categorical relations
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["fruit"],
        "is_a",
        agents[3]  # nutritionist
    )
    
    semantica.propose_relation(
        primitive_dict["orange"],
        primitive_dict["fruit"],
        "is_a",
        agents[3]  # nutritionist
    )
    
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["dessert"],
        "can_be_used_as",
        agents[4]  # chef
    )
    
    # Conceptual relations
    semantica.propose_relation(
        primitive_dict["fruit"],
        primitive_dict["health"],
        "promotes",
        agents[3]  # nutritionist
    )
    
    semantica.propose_relation(
        primitive_dict["fruit"],
        primitive_dict["nutrition"],
        "provides",
        agents[3]  # nutritionist
    )
    
    # Seasonal relations
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["autumn"],
        "associated_with",
        agents[0]
    )
    
    semantica.propose_relation(
        primitive_dict["orange"],
        primitive_dict["summer"],
        "associated_with",
        agents[0]
    )
    
    # Size relations
    semantica.propose_relation(
        primitive_dict["apple"],
        primitive_dict["small"],
        "has_typical_size",
        agents[1]
    )
    
    semantica.propose_relation(
        primitive_dict["orange"],
        primitive_dict["small"],
        "has_typical_size",
        agents[1]
    )
    
    # Create an animated visualization with more frames for complexity
    output_path = semantica.animate_convergence(num_frames=500)
    return semantica, output_path

# Run the demonstration
if __name__ == "__main__":
    # Specify the output directory
    output_dir = r"C:\Users\Erich Curtis\Desktop\All Python\Semantica\Data\Output"
    
    # Run the demo
    demo_graph, animation_path = semantic_convergence_demo(output_dir)
    print(f"Animation created at: {animation_path}")

# Phase 2

In [1]:
# Phase 2: Dynamic Data Integration and Enhanced Visualization
# This phase integrates ConceptNet datasets into the visualization pipeline, allowing dynamic sampling and ensuring at least 100 matching words between datasets.
import sys
sys.path.append(r'C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Py Scripts')  # or the correct relative path from your notebook to Py Scripts

import pandas as pd
from conceptnet_processor import ConceptNetProcessor_v2

english_conceptnet = pd.read_csv('../Data/Input/conceptnet-assertions-5.7.0.en.tsv', sep='\t', names=['assertion', 'rel', 'start', 'end', 'meta'], header=None)
german_conceptnet = pd.read_csv('../Data/Input/conceptnet-assertions-5.7.0.de.tsv', sep='\t', names=['assertion', 'rel', 'start', 'end', 'meta'], header=None)

max_concepts = 100  # Maximum number of concepts to sample
# Load datasets
processor = ConceptNetProcessor_v2(english_data=english_conceptnet, german_data=german_conceptnet)
processor.build_semantic_graph(max_concepts=max_concepts, sample_size=0.05)

# After building the semantic graph
print(f'Number of nodes in the semantic graph: {len(processor.semantic_graph.nodes())}')
print(f'Sample nodes: {list(processor.semantic_graph.nodes())[:10]}')

# Check the intersection of nodes
matching_words = set(processor.semantic_graph.nodes()) & set(processor.semantic_graph.nodes())
print(f'Number of matching words: {len(matching_words)}')
print(f'Sample matching words: {list(matching_words)[:10]}')

ConceptNetProcessor initialized
Building semantic graph from ConceptNet data...
Processing 3423004 English ConceptNet assertions...
Will sample 171150 English assertions
Processing 1078946 German ConceptNet assertions...
Will sample 53947 German assertions
Sampling 171150 assertions from 3423004 en assertions


Processing en assertions: 100%|██████████| 171150/171150 [00:05<00:00, 30728.03it/s]



Sampling 53947 assertions from 1078946 de assertions


Processing de assertions: 100%|██████████| 53947/53947 [00:01<00:00, 32937.14it/s]

Limiting graph to top 100 concepts...
Semantic graph built with 100 nodes and 708 edges
Inferring semantic categories...
Inferred categories: {'generic': 100}
Number of nodes in the semantic graph: 100
Sample nodes: ['attribute', 'slang', 'botany', 'fish', 'england', 'uk', 'chemistry', 'n', 'wn', 'communication']
Number of matching words: 100
Sample matching words: ['attribute', 'fish', 'botany', 'slang', 'england', 'uk', 'chemistry', 'n', 'wn', 'communication']


In [2]:
# Ensure at least 100 matching words
matching_words = set(processor.semantic_graph.nodes()) & set(processor.semantic_graph.nodes())
if len(matching_words) < max_concepts:
    raise ValueError("Not enough matching words between datasets.")

In [5]:
output_dir = r"C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output"
# Update primitives and relations dynamically
semantica = SemanticaGraph(output_dir)

# Create a dictionary to store primitives for easy access
primitive_dict = {}

# Add primitives from ConceptNet
for word in matching_words:
    primitive = SemanticaPrimitive(value=word)
    semantica.add_primitive(primitive)
    primitive_dict[word] = primitive

# Create agents to work with the ConceptNet data
agents = [
    SemanticAgent("conceptnet_expert", {"translation": 0.9, "similarity": 0.8, "categorization": 0.9, "exploration": 0.7}),
    SemanticAgent("linguistic_analyst", {"similarity": 0.9, "exploration": 0.8, "categorization": 0.7}),
    SemanticAgent("knowledge_integrator", {"categorization": 0.9, "exploration": 0.9}),
    SemanticAgent("semantic_explorer", {"exploration": 1.0, "similarity": 0.7})
]

# Add agents to the semantica graph
for agent in agents:
    semantica.add_agent(agent)

# Extract meaningful relations from the ConceptNet processor's semantic graph
print("Adding relations based on ConceptNet data...")
relation_count = 0
relation_types = {"IsA", "RelatedTo", "AtLocation", "HasProperty", "UsedFor", "CapableOf", "HasA", "PartOf"}

# Add relations based on the processor's semantic graph edges
for source, target, data in processor.semantic_graph.edges(data=True):
    if source in primitive_dict and target in primitive_dict:
        rel_type = data.get('relation', 'RelatedTo')
        rel_short = rel_type.split('/')[-1] if '/' in rel_type else rel_type
        
        # Choose an appropriate agent based on relation type
        if rel_short in ["IsA", "PartOf"]:
            chosen_agent = agents[2]  # knowledge_integrator for categorization
        elif rel_short in ["RelatedTo", "SimilarTo"]:
            chosen_agent = agents[1]  # linguistic_analyst for similarity
        elif rel_short in ["HasProperty", "HasA"]:
            chosen_agent = agents[0]  # conceptnet_expert for properties
        else:
            chosen_agent = agents[3]  # semantic_explorer for other relations
        
        # Propose the relation
        semantica.propose_relation(
            primitive_dict[source],
            primitive_dict[target],
            rel_short,
            chosen_agent
        )
        relation_count += 1
        
        # Limit the number of relations to avoid overwhelming visualization
        if relation_count >= 150:
            break

# Add some additional cross-relations to enhance connectivity
print(f"Added {relation_count} relations from ConceptNet data")

# Sample some common concepts to create additional relations
common_concepts = [word for word in matching_words if len(word) < 10][:50]
for i in range(30):  # Add 30 additional exploration relations
    if len(common_concepts) >= 2:
        source_word = random.choice(common_concepts)
        target_word = random.choice(common_concepts)
        if source_word != target_word and source_word in primitive_dict and target_word in primitive_dict:
            semantica.propose_relation(
                primitive_dict[source_word],
                primitive_dict[target_word],
                "exploration",
                agents[3]
            )

# Print statistics about the graph we've built
print(f"Created graph with {len(semantica.primitives)} primitives and {len(semantica.relations)} relations")
print(f"Convergence graph has {semantica.convergence_graph.number_of_nodes()} nodes and {semantica.convergence_graph.number_of_edges()} edges")

# Create enhanced animation
output_path = semantica.animate_convergence(num_frames=100)
print(f"Enhanced animation created at: {output_path}")

Adding relations based on ConceptNet data...
Added 150 relations from ConceptNet data
Created graph with 100 primitives and 179 relations
Convergence graph has 96 nodes and 177 edges
Number of primitives: 100
Number of relations: 179
Number of nodes in convergence graph: 96
Number of edges in convergence graph: 177
Animation saved to C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output\semantic_convergence_f43c0ee1-bf84-4d93-b362-26e84bcc05ef.gif
Enhanced animation created at: C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output\semantic_convergence_f43c0ee1-bf84-4d93-b362-26e84bcc05ef.gif
Animation saved to C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output\semantic_convergence_f43c0ee1-bf84-4d93-b362-26e84bcc05ef.gif
Enhanced animation created at: C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output\semantic_convergence_f43c0ee1-bf84-4d93-b362-26e84bcc05ef.gif


# Phase 3: Enhanced Visualization Dynamics

In this phase, we'll enhance the visualization with:
1. Improved force-directed layouts with multiple layout options
2. Enhanced visual aesthetics with customizable color schemes
3. Animation transitions between semantic states
4. Interactive elements for a more engaging experience

These enhancements will create more visually compelling and informative representations of semantic convergence.

In [4]:
# Phase 3: Enhanced Layout Dynamics
# This extends the SemanticaGraph visualization capabilities with enhanced layout options

import colorsys
from matplotlib.colors import LinearSegmentedColormap

# Define enhanced layout methods to extend SemanticaGraph
class EnhancedLayoutManager:
    """Provides enhanced layout algorithms for semantic graph visualization"""
    
    def __init__(self, graph: SemanticaGraph):
        self.graph = graph
        self.layout_history = []
        self.color_schemes = {
            'nebula': plt.cm.plasma,
            'ocean': plt.cm.viridis,
            'fire': plt.cm.magma,
            'forest': plt.cm.YlGn,
            'cosmic': self._create_cosmic_colormap()
        }
        self.current_scheme = 'nebula'
    
    def _create_cosmic_colormap(self):
        """Creates a custom colormap for cosmic theme"""
        # Deep space blue to bright star gold
        colors = [(0.05, 0.05, 0.2), (0.3, 0.2, 0.5), (0.7, 0.5, 0.9), (1, 0.9, 0.6)]
        return LinearSegmentedColormap.from_list('cosmic', colors)
    
    def get_layout(self, layout_type='force_directed', **kwargs):
        """Generate a layout based on specified algorithm"""
        if layout_type == 'force_directed':
            return nx.spring_layout(self.graph.convergence_graph, **kwargs)
        elif layout_type == 'circular':
            return nx.circular_layout(self.graph.convergence_graph)
        elif layout_type == 'spectral':
            return nx.spectral_layout(self.graph.convergence_graph)
        elif layout_type == 'spiral':
            return self._create_spiral_layout()
        elif layout_type == 'semantic_cluster':
            return self._create_semantic_cluster_layout()
        else:
            return nx.spring_layout(self.graph.convergence_graph, **kwargs)
    
    def _create_spiral_layout(self):
        """Places nodes in a spiral pattern based on semantic energy"""
        layout = {}
        nodes = list(self.graph.convergence_graph.nodes())
        
        # Sort nodes by semantic energy
        nodes.sort(key=lambda n: self.graph.primitives[n].semantic_energy)
        
        # Create spiral coordinates
        for i, node in enumerate(nodes):
            theta = i * 0.5
            r = 0.1 + i * 0.05
            x = r * np.cos(theta)
            y = r * np.sin(theta)
            layout[node] = np.array([x, y])
            
        return layout
    
    def _create_semantic_cluster_layout(self):
        """Clusters nodes based on relation types and language"""
        G = self.graph.convergence_graph
        layout = {}
        
        # Group nodes by language and primitive type
        language_groups = {}
        type_groups = {}
        
        for node in G.nodes():
            primitive = self.graph.primitives[node]
            lang = primitive.language
            prim_type = primitive.type
            
            if lang not in language_groups:
                language_groups[lang] = []
            language_groups[lang].append(node)
            
            if prim_type not in type_groups:
                type_groups[prim_type] = []
            type_groups[prim_type].append(node)
        
        # Position clusters
        num_langs = len(language_groups)
        for i, (lang, nodes) in enumerate(language_groups.items()):
            # Create a cluster position
            cluster_x = np.cos(2 * np.pi * i / num_langs)
            cluster_y = np.sin(2 * np.pi * i / num_langs)
            
            # Position nodes within cluster
            for j, node in enumerate(nodes):
                angle = 2 * np.pi * j / len(nodes)
                radius = 0.2
                x = cluster_x + radius * np.cos(angle)
                y = cluster_y + radius * np.sin(angle)
                layout[node] = np.array([x, y])
        
        return layout
    
    def morph_between_layouts(self, start_layout, end_layout, steps=10):
        """Create a sequence of layouts morphing from start to end"""
        layout_sequence = []
        
        for step in range(steps + 1):
            alpha = step / steps  # Interpolation parameter
            current_layout = {}
            
            for node in start_layout:
                # Linear interpolation between start and end positions
                start_pos = start_layout[node]
                end_pos = end_layout[node]
                current_pos = start_pos * (1 - alpha) + end_pos * alpha
                current_layout[node] = current_pos
                
            layout_sequence.append(current_layout)
            
        return layout_sequence
    
    def set_color_scheme(self, scheme_name):
        """Set the current color scheme for visualization"""
        if scheme_name in self.color_schemes:
            self.current_scheme = scheme_name
            return True
        return False
    
    def get_node_colors(self, attribute='semantic_energy'):
        """Get node colors based on a primitive attribute"""
        values = [getattr(self.graph.primitives[node], attribute) for node in self.graph.convergence_graph.nodes()]
        colormap = self.color_schemes[self.current_scheme]
        return colormap(np.array(values) / max(values) if values else [0])
    
    def get_edge_colors(self, attribute='confidence'):
        """Get edge colors based on relation attribute"""
        edge_attrs = []
        for u, v in self.graph.convergence_graph.edges():
            edge_attrs.append(self.graph.convergence_graph[u][v].get(attribute, 0.1))
            
        colormap = self.color_schemes[self.current_scheme]
        return colormap(np.array(edge_attrs) / max(edge_attrs) if edge_attrs else [0])

In [5]:
# Phase 3: Implementation of the enhanced animation
# This enhances SemanticaGraph with a new animation method using the EnhancedLayoutManager

# Extend SemanticaGraph with enhanced animation capabilities
def enhanced_animate_convergence(semantic_graph, num_frames=120, layout_sequence=['force_directed', 'semantic_cluster', 'spiral'], color_scheme='cosmic'):
    """Create an enhanced animated visualization with layout transitions and improved visual aesthetics"""
    
    # Create layout manager
    layout_manager = EnhancedLayoutManager(semantic_graph)
    layout_manager.set_color_scheme(color_scheme)
    
    fig, ax = plt.subplots(figsize=(16, 12), facecolor='black')
    plt.style.use('dark_background')
    
    # Precompute layouts for each type
    layouts = {}
    for layout_type in set(layout_sequence):
        layouts[layout_type] = layout_manager.get_layout(layout_type)
    
    # Determine transitions
    transitions = []
    frames_per_layout = num_frames // len(layout_sequence)
    morph_frames = min(15, frames_per_layout // 3)  # Use at most 1/3 of frames for transitions
    
    # Create morphing sequences between layouts
    current_frame = 0
    current_layouts = []
    
    for i in range(len(layout_sequence)):
        current_layout = layout_sequence[i]
        next_layout = layout_sequence[(i + 1) % len(layout_sequence)]
        
        # Add stable frames with current layout
        stable_frames = frames_per_layout - morph_frames
        for _ in range(stable_frames):
            current_layouts.append(layouts[current_layout])
            current_frame += 1
        
        # Add morphing frames to next layout
        morph_sequence = layout_manager.morph_between_layouts(
            layouts[current_layout], 
            layouts[next_layout], 
            steps=morph_frames
        )
        
        current_layouts.extend(morph_sequence)
        current_frame += len(morph_sequence)
    
    def update(frame):
        ax.clear()
        ax.set_facecolor('black')
        plt.title(f"Semantic Convergence - {layout_sequence[min(frame // frames_per_layout, len(layout_sequence)-1)]} Layout", 
                  color='white', fontsize=18, fontweight='bold')
        
        # Get the current layout
        pos = current_layouts[frame % len(current_layouts)]
        
        # Periodically update primitive attributes to create dynamic visuals
        if frame % 5 == 0:
            # Update semantic energy with small random fluctuations
            for node_id in semantic_graph.convergence_graph.nodes():
                primitive = semantic_graph.primitives[node_id]
                energy_delta = random.uniform(-0.02, 0.05)  # Small random change
                primitive.semantic_energy = max(0.1, min(1.0, primitive.semantic_energy + energy_delta))
        
        # Get node colors using the layout manager
        node_colors = layout_manager.get_node_colors()
        node_sizes = [100 + semantic_graph.primitives[node].semantic_energy * 800 
                     for node in semantic_graph.convergence_graph.nodes()]
        
        # Get edge colors using the layout manager
        edge_colors = layout_manager.get_edge_colors()
        edge_weights = [semantic_graph.convergence_graph[u][v].get('confidence', 0.1) * 5
                      for (u, v) in semantic_graph.convergence_graph.edges()]
        
        # Draw nodes with enhanced glow effect
        for i, node in enumerate(semantic_graph.convergence_graph.nodes()):
            node_size = node_sizes[i]
            node_color = node_colors[i]
            node_alpha = 0.9
            
            # Draw glow effect (larger, more transparent circle)
            glow_size = node_size * 1.5
            glow_alpha = 0.3
            nx.draw_networkx_nodes(
                semantic_graph.convergence_graph,
                pos,
                nodelist=[node],
                node_size=glow_size,
                node_color=[node_color],
                alpha=glow_alpha
            )
            
            # Draw main node
            nx.draw_networkx_nodes(
                semantic_graph.convergence_graph,
                pos,
                nodelist=[node],
                node_size=node_size,
                node_color=[node_color],
                alpha=node_alpha
            )
        
        # Draw edges with enhanced styling
        for i, (u, v) in enumerate(semantic_graph.convergence_graph.edges()):
            # Create curved edges with varying arc
            relation_type = semantic_graph.convergence_graph[u][v].get('relation_type', '')
            # Different curve styles for different relation types
            if 'translation' in relation_type:
                connectionstyle = 'arc3,rad=0.2'
            elif 'is_a' in relation_type or 'IsA' in relation_type:
                connectionstyle = 'arc3,rad=-0.2'
            else:
                connectionstyle = 'arc3,rad=0.1'
                
            nx.draw_networkx_edges(
                semantic_graph.convergence_graph,
                pos,
                edgelist=[(u, v)],
                width=edge_weights[i],
                edge_color=[edge_colors[i]],
                alpha=0.7,
                arrows=True,
                arrowsize=10,
                connectionstyle=connectionstyle
            )
        
        # Draw better labels with backgrounds
        for node, (x, y) in pos.items():
            primitive = semantic_graph.primitives[node]
            label = primitive.value
            energy = primitive.semantic_energy
            
            # Only show labels for nodes with higher energy to reduce clutter
            if energy > 0.3 or random.random() < 0.2:  # Show some random labels as well
                # Create text with shadow/outline effect for better readability
                label_color = 'white'
                bbox_props = dict(boxstyle="round,pad=0.3", alpha=0.7, ec="none", fc="black")
                font_size = 9 + energy * 3  # Scale font size with energy
                ax.text(x, y, label, size=font_size, color=label_color, 
                        ha='center', va='center', bbox=bbox_props, zorder=5)
        
        # Add information display
        total_semantic_energy = sum(p.semantic_energy for p in semantic_graph.primitives.values())
        ax.text(0.02, 0.98, f"Frame: {frame} / {num_frames}", transform=ax.transAxes, 
                color='white', fontsize=10, verticalalignment='top', fontweight='bold')
        ax.text(0.02, 0.94, f"Semantic Energy: {total_semantic_energy:.2f}", transform=ax.transAxes, 
                color='white', fontsize=10, verticalalignment='top')
        ax.text(0.02, 0.90, f"Nodes: {semantic_graph.convergence_graph.number_of_nodes()}", 
                transform=ax.transAxes, color='white', fontsize=10, verticalalignment='top')
        ax.text(0.02, 0.86, f"Relations: {semantic_graph.convergence_graph.number_of_edges()}", 
                transform=ax.transAxes, color='white', fontsize=10, verticalalignment='top')
        
        # Add color scheme info
        ax.text(0.98, 0.98, f"Theme: {color_scheme}", transform=ax.transAxes, 
                color='white', fontsize=10, ha='right', va='top')
        
        plt.axis('off')
        
    # Create animation
    anim = animation.FuncAnimation(fig, update, frames=num_frames, interval=100, blit=False)
    
    # Save the animation
    output_path = os.path.join(semantic_graph.output_dir, f'enhanced_semantic_convergence_{uuid.uuid4()}.gif')
    anim.save(output_path, writer='pillow', fps=10, dpi=100)
    plt.close(fig)
    
    print(f"Enhanced animation saved to {output_path}")
    return output_path

In [ ]:
# Test our enhanced animation with existing semantic graph
# Let's create a demonstration with different color schemes and layouts

def phase3_visualization_demo():
    # Use the existing semantic graph from Phase 2 or create a new one
    output_dir = r"C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output"
    
    # First, create a small sample semantic graph if needed
    semantica = SemanticaGraph(output_dir)
    
    # Check if we already have a populated graph from Phase 2
    if semantica.convergence_graph.number_of_nodes() < 10:
        print("Creating new semantic graph for demonstration...")
        # Create a simple demonstration graph
        demo_primitives = [
            SemanticaPrimitive("concept", primitive_type="abstract", language="english"),
            SemanticaPrimitive("idea", primitive_type="abstract", language="english"),
            SemanticaPrimitive("thought", primitive_type="abstract", language="english"),
            SemanticaPrimitive("knowledge", primitive_type="abstract", language="english"),
            SemanticaPrimitive("understanding", primitive_type="abstract", language="english"),
            SemanticaPrimitive("wisdom", primitive_type="abstract", language="english"),
            SemanticaPrimitive("meaning", primitive_type="abstract", language="english"),
            SemanticaPrimitive("language", primitive_type="system", language="english"),
            SemanticaPrimitive("communication", primitive_type="process", language="english"),
            SemanticaPrimitive("symbols", primitive_type="concrete", language="english"),
            SemanticaPrimitive("words", primitive_type="concrete", language="english"),
            SemanticaPrimitive("grammar", primitive_type="system", language="english"),
            SemanticaPrimitive("syntax", primitive_type="system", language="english"),
            SemanticaPrimitive("semantics", primitive_type="field", language="english"),
            SemanticaPrimitive("Gedanke", primitive_type="abstract", language="german"),
            SemanticaPrimitive("Idee", primitive_type="abstract", language="german"),
            SemanticaPrimitive("Wissen", primitive_type="abstract", language="german"),
            SemanticaPrimitive("Sprache", primitive_type="system", language="german"),
            SemanticaPrimitive("Kommunikation", primitive_type="process", language="german"),
            SemanticaPrimitive("Wörter", primitive_type="concrete", language="german")
        ]
        
        # Add primitives to graph
        primitive_dict = {}
        for p in demo_primitives:
            semantica.add_primitive(p)
            primitive_dict[p.value] = p
        
        # Create agents
        agents = [
            SemanticAgent("linguist", {"translation": 0.9, "similarity": 0.8}),
            SemanticAgent("philosopher", {"conceptual": 0.9, "similarity": 0.7})
        ]
        
        for agent in agents:
            semantica.add_agent(agent)
        
        # Add relations
        # Translations
        semantica.propose_relation(primitive_dict["thought"], primitive_dict["Gedanke"], "translation", agents[0])
        semantica.propose_relation(primitive_dict["idea"], primitive_dict["Idee"], "translation", agents[0])
        semantica.propose_relation(primitive_dict["knowledge"], primitive_dict["Wissen"], "translation", agents[0])
        semantica.propose_relation(primitive_dict["language"], primitive_dict["Sprache"], "translation", agents[0])
        semantica.propose_relation(primitive_dict["communication"], primitive_dict["Kommunikation"], "translation", agents[0])
        semantica.propose_relation(primitive_dict["words"], primitive_dict["Wörter"], "translation", agents[0])
        
        # Conceptual relations
        semantica.propose_relation(primitive_dict["concept"], primitive_dict["idea"], "similar_to", agents[1])
        semantica.propose_relation(primitive_dict["concept"], primitive_dict["thought"], "similar_to", agents[1])
        semantica.propose_relation(primitive_dict["language"], primitive_dict["communication"], "enables", agents[0])
        semantica.propose_relation(primitive_dict["language"], primitive_dict["words"], "contains", agents[0])
        semantica.propose_relation(primitive_dict["language"], primitive_dict["grammar"], "contains", agents[0])
        semantica.propose_relation(primitive_dict["language"], primitive_dict["syntax"], "contains", agents[0])
        semantica.propose_relation(primitive_dict["meaning"], primitive_dict["semantics"], "relates_to", agents[1])
        semantica.propose_relation(primitive_dict["understanding"], primitive_dict["knowledge"], "requires", agents[1])
        semantica.propose_relation(primitive_dict["wisdom"], primitive_dict["knowledge"], "builds_on", agents[1])
        semantica.propose_relation(primitive_dict["grammar"], primitive_dict["syntax"], "includes", agents[0])
        
        # Add a few more connections to create interesting structure
        for _ in range(10):
            source = random.choice(demo_primitives)
            target = random.choice(demo_primitives)
            if source != target:
                semantica.propose_relation(source, target, "relates_to", random.choice(agents))
    
    # Now create animations with different color schemes and layouts
    color_schemes = ['nebula', 'cosmic', 'ocean', 'fire', 'forest']
    layout_combinations = [
        ['force_directed', 'semantic_cluster', 'spiral'],
        ['semantic_cluster', 'circular', 'spectral'],
        ['spiral', 'force_directed', 'circular'],
        ['circular', 'spiral', 'force_directed', 'semantic_cluster']
    ]
    
    # Create first demonstration with cosmic color scheme
    print("Creating enhanced visualization with cosmic theme...")
    cosmic_path = enhanced_animate_convergence(
        semantica, 
        num_frames=120, 
        layout_sequence=layout_combinations[0],
        color_scheme='cosmic'
    )
    
    # Create second demonstration with different layout and color scheme
    print("Creating enhanced visualization with nebula theme...")
    nebula_path = enhanced_animate_convergence(
        semantica, 
        num_frames=120, 
        layout_sequence=layout_combinations[1],
        color_scheme='nebula'
    )
    
    return cosmic_path, nebula_path

# Run the Phase 3 demonstration
cosmic_path, nebula_path = phase3_visualization_demo()
print(f"\nVisualization demo completed!\nCosmic theme: {cosmic_path}\nNebula theme: {nebula_path}")

Creating new semantic graph for demonstration...
Creating enhanced visualization with cosmic theme...
Enhanced animation saved to C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output\enhanced_semantic_convergence_e614138e-c3db-458a-a3d2-b5c818f863b7.gif
Creating enhanced visualization with nebula theme...
Enhanced animation saved to C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output\enhanced_semantic_convergence_4ba5d1eb-b16a-4dba-a6f3-bb2eeeb6e455.gif

Visualization demo completed!
Cosmic theme: C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output\enhanced_semantic_convergence_e614138e-c3db-458a-a3d2-b5c818f863b7.gif
Nebula theme: C:\Users\erich\OneDrive\Documents\Python Projects\Semantica\Semantica\Data\Output\enhanced_semantic_convergence_4ba5d1eb-b16a-4dba-a6f3-bb2eeeb6e455.gif
